# Neural Network - Training on Binary Lists

This notebook shows how to set up a training scenario using a neural network.
It will be used to feedforward a list of bits and convert it to a number.

## Code Implementation

### Libraries and Helper Functions

First, we need to import `NeuralNetwork`.

In [1]:
from datetime import datetime

import numpy as np

from neural_network.math.activation_functions import LinearActivation, SigmoidActivation
from neural_network.neural_network import NeuralNetwork
from neural_network.nn.layer import HiddenLayer, InputLayer, OutputLayer

We can define some helper functions to analyse the results from the training.

In [2]:
def generate_time_msg() -> str:
    """
    Get message prefix with current datetime.
    """
    time_msg = f"[{datetime.now().strftime('%d-%m-%G | %H:%M:%S')}]"
    return time_msg


def print_system_msg(msg: str) -> None:
    """
    Print a message to the terminal.

    Parameters:
        msg (str): Message to print
    """
    print(f"{generate_time_msg()} {msg}")


def print_flushed_msg(msg: str) -> None:
    """
    Print a flushed message to the terminal.

    Parameters:
        msg (str): Message to print
    """
    print(f"\r{generate_time_msg()} {msg}", flush=True, end="")


def calculate_rms(errors: list[float]) -> float:
    """
    Calculate RMS from errors.

    Parameters:
        errors (list[float]): Errors from neural network training

    Returns:
        rms (float): RMS from errors
    """
    squared = np.square(errors)
    mean = np.average(squared)
    rms = np.sqrt(mean)
    return rms

### Creating Methods to Generate Training Data

We will be using 8-bit numbers to train the neural network.
We can use the following bit map to convert numbers between integers and byte lists.

In [3]:
NUM_BITS = 8
BIT_MAP = np.array([2 ** (NUM_BITS - (i + 1)) for i in range(NUM_BITS)])
print_system_msg(f"Bit map: {BIT_MAP}")


def num_to_byte_list(num: int) -> list[int]:
    """
    Convert a number to a list of bits.

    Parameters:
        num (int): Number to convert

    Returns:
        byte_list (list[int]): Number represented as list of bits
    """
    _num_bin = bin(num)
    _num_bytes = _num_bin[2:]
    _padding = [0] * (NUM_BITS - len(_num_bytes))
    byte_list = _padding + [int(b) for b in _num_bytes]
    return byte_list

[11-05-2024 | 21:18:15] Bit map: [128  64  32  16   8   4   2   1]


With 8 bits, we can generate numbers between 0-255.
The neural network outputs numbers between 0-1 and therefore we need to map the values accordingly.

In [4]:
IN_LIMS = [0, 255]
OUT_LIMS = [0, 1]


def map_val(x: float, in_min: float, in_max: float, out_min: float, out_max: float) -> float:
    """
    Map a value from an input range to an output range.

    Parameters:
        x (float): Number to map to new range
        in_min (float): Lower bound of original range
        in_max (float): Upper bound of original range
        out_min (float): Lower bound of new range
        out_max (float): Upper bound of new range

    Returns:
        y (float): Number mapped to new range
    """
    y = (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min
    return y


def training_data_from_num(num: int) -> tuple[list[int], list[float]]:
    """
    Generate byte list and mapped number from a number to use in training.

    Parameters:
        num (int): Number to use for training data

    Returns:
        training_data (tuple[list[int], list[float]]): Input and expected output
    """
    _byte_list = np.array(num_to_byte_list(num))
    _mapped_num = map_val(num, IN_LIMS[0], IN_LIMS[1], OUT_LIMS[0], OUT_LIMS[1])
    training_data = (_byte_list, _mapped_num)
    return training_data

### Creating the Neural Network

The number of inputs for the neural network is `NUM_BITS`.
The number of outputs is 1 as we will be converting a float to a number in our original range.


In [5]:
hidden_layer_sizes = [3]
lr = 0.2

input_layer = InputLayer(size=NUM_BITS, activation=LinearActivation)
hidden_layers = [
    HiddenLayer(size=size, activation=SigmoidActivation, weights_range=[-1, 1], bias_range=[-1, 1])
    for size in hidden_layer_sizes
]
output_layer = OutputLayer(size=1, activation=SigmoidActivation, weights_range=[-1, 1], bias_range=[-1, 1])

nn = NeuralNetwork(layers=[input_layer, *hidden_layers, output_layer], lr=lr)

### Running the Algorithm

To train the neural network, we will select a random number and train the neural network with the corresponding byte list and expected output.

First, we need to create the dataset.
We will generate a training dataset with the specified size.
We will then allocate a subset of this data to be used for testing.

In [6]:
dataset_size = 30000
test_dataset_ratio = 0.2

print_system_msg(f"Creating dataset with {dataset_size} inputs and expected outputs...")
dataset_inputs = []
dataset_outputs = []

for _ in range(dataset_size):
    random_num = np.random.randint(low=IN_LIMS[0], high=(IN_LIMS[1] + 1))
    training_input, expected_output = training_data_from_num(random_num)

    dataset_inputs.append(training_input)
    dataset_outputs.append(expected_output)

train_dataset_input = dataset_inputs[: int(dataset_size * (1 - test_dataset_ratio))]
train_dataset_output = dataset_outputs[: int(dataset_size * (1 - test_dataset_ratio))]

test_dataset_input = dataset_inputs[int(dataset_size * (1 - test_dataset_ratio)) :]
test_dataset_output = dataset_outputs[int(dataset_size * (1 - test_dataset_ratio)) :]
print_system_msg(f"Training dataset: {len(train_dataset_input)} items | Test dataset: {len(test_dataset_input)} items")

[11-05-2024 | 21:18:15] Creating dataset with 30000 inputs and expected outputs...
[11-05-2024 | 21:18:15] Training dataset: 24000 items | Test dataset: 6000 items


Now we can begin the training process with the training inputs and expected outputs.

In [7]:
num_iters = len(train_dataset_input)
print_system_msg(f"Beginning training with {num_iters} data points...")

begin_time = datetime.now()

for i in range(num_iters):
    errors = nn.train(train_dataset_input[i], [train_dataset_output[i]])
    rms = calculate_rms(errors)
    print_flushed_msg(f"{i+1} / {num_iters} -> RMS: {rms:.4f}")

end_time = datetime.now()
print_flushed_msg(f"{num_iters} / {num_iters} -> Training complete! Final error: {rms:.4f}\n")

delta_time = (end_time - begin_time).total_seconds()
print_system_msg(f"Training took {delta_time:.1f}s.")

[11-05-2024 | 21:18:15] Beginning training with 24000 data points...
[11-05-2024 | 21:18:28] 24000 / 24000 -> Training complete! Final error: 0.0447
[11-05-2024 | 21:18:28] Training took 13.0s.


We can then test the neural network against some inputs and expected outputs to check its accuracy.

In [8]:
num_iters = len(test_dataset_input)
outputs = []
print_system_msg(f"Beginning testing with {num_iters} data points...")

begin_time = datetime.now()

for i in range(num_iters):
    print_flushed_msg(f"{i+1} / {num_iters} -> Testing...")
    output = nn.feedforward(test_dataset_input[i])[0]
    outputs.append(output)

end_time = datetime.now()
print_flushed_msg(f"{num_iters} / {num_iters} -> Testing complete!\n")

delta_time = (end_time - begin_time).total_seconds()
print_system_msg(f"Testing took {delta_time:.1f}s.")
print_system_msg(f"Number of calculations per second: {int(num_iters / delta_time)}")

[11-05-2024 | 21:18:28] Beginning testing with 6000 data points...
[11-05-2024 | 21:18:30] 6000 / 6000 -> Testing complete!
[11-05-2024 | 21:18:30] Testing took 1.9s.
[11-05-2024 | 21:18:30] Number of calculations per second: 3212


In [9]:
errors = np.array(test_dataset_output) - np.array(outputs)
mapped_errors = map_val(errors, OUT_LIMS[0], OUT_LIMS[1], IN_LIMS[0], IN_LIMS[1])

avg_error = np.average(mapped_errors)
percentage_error = np.abs(avg_error) / IN_LIMS[1]
print_system_msg(f"Average error: {avg_error:.2f} \t| Percentage error: {percentage_error:.4f}%")

[11-05-2024 | 21:18:30] Average error: 0.59 	| Percentage error: 0.0023%
